In [6]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy.interpolate as sci

import gala.coordinates as gc
from pyia import GaiaData
import galstreams
import sys
sys.path.append('../')
import cats.photometry as phot
from cats.data import make_astro_photo_joined_data

In [7]:
mws = galstreams.MWStreams(verbose=False, implement_Off=True)

Initializing galstreams library from master_log... 


# GD-1

In [6]:
name = 'GD-1'
ms_name = 'GD-1-PB18'
g = GaiaData(f'../data/GaiaDR3-{name}-all.fits')
p = phot.PS1Phot('../data/PS1DR2-GD-1_xm.fits')
gs = mws[ms_name]

In [8]:
joined = make_astro_photo_joined_data(g, p, gs)
joined.write(f'../data/joined-{name}.fits', overwrite=True)

/mnt/home/apricewhelan/anaconda3/lib/python3.9/site-packages/astropy/io/fits/card.py:264: VerifyWarning: Keyword name 'band_names' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
  warnings.warn(



# Pal 5

In [8]:
name = 'Pal5'
ms_name = 'Pal5-PW19'
g = GaiaData(f'../data/GaiaDR3-{name}-all.fits')
p = phot.PS1Phot('../data/PS1DR2-Pal5_xm.fits')
gs = mws[ms_name]

FileNotFoundError: [Errno 2] No such file or directory: '../data/GaiaDR3-Pal5-all.fits'

In [11]:
joined = make_astro_photo_joined_data(g, p, gs)
joined.write(f'../data/joined-{name}.fits', overwrite=True)

/mnt/home/apricewhelan/anaconda3/lib/python3.9/site-packages/astropy/io/fits/card.py:264: VerifyWarning: Keyword name 'band_names' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
  warnings.warn(



# Jhelum

In [19]:
tbl_gaia = at.Table.read('../data/GaiaDR3-Jhelum-all.fits')
tbl_des = at.Table.read('../data/Jhelum/jhelum_desy6.fits')

c_des = coord.SkyCoord(tbl_des['RA']*u.deg, tbl_des['DEC']*u.deg)
c_gaia = coord.SkyCoord(tbl_gaia['ra']*u.deg, tbl_gaia['dec']*u.deg)

In [20]:
idx, sep, _ = c_gaia.match_to_catalog_sky(c_des)

In [27]:
sep_mask = sep < 3*u.arcsec
tbl_des_sub = tbl_des[idx[sep_mask]]
tbl_des_sub['source_id'] = tbl_gaia[sep_mask]['source_id']

In [29]:
name = 'Jhelum'
ms_name = 'Jhelum-b-B19'
g = GaiaData(f'../data/GaiaDR3-{name}-all.fits')
p = phot.DESY6Phot(tbl_des_sub)
gs = mws[ms_name]

In [30]:
joined = make_astro_photo_joined_data(g, p, gs)
joined.write(f'../data/joined-{name}.fits', overwrite=True)

/mnt/home/apricewhelan/anaconda3/lib/python3.9/site-packages/astropy/io/fits/card.py:264: VerifyWarning: Keyword name 'band_names' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
  warnings.warn(



# PS1-A

In [31]:
name = 'PS1-A'
ms_name = 'PS1-A-B16'
g = GaiaData(f'../data/GaiaDR3-{name}-all.fits')
p = phot.PS1Phot('../data/PS1DR2-PS1-A_xm.fits')
gs = mws[ms_name]

In [32]:
joined = make_astro_photo_joined_data(g, p, gs)
joined.write(f'../data/joined-{name}.fits', overwrite=True)

/mnt/home/apricewhelan/anaconda3/lib/python3.9/site-packages/astropy/io/fits/card.py:264: VerifyWarning: Keyword name 'band_names' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
  warnings.warn(



# Fjorm

In [34]:
name = 'Fjorm-M68'
ms_name = 'M68-P19'
g = GaiaData(f'../data/GaiaDR3-{name}-all.fits')
p = phot.GaiaDR3Phot(g.data)
gs = mws[ms_name]

In [35]:
joined = make_astro_photo_joined_data(g, p, gs)
joined.write(f'../data/joined-{name}.fits', overwrite=True)